In [1]:
%run ../talktools.py

# Stellar Spectroscopy

Astro 128/256 (UC Berkeley, 2024)

## Motivation
This lab is all about stellar spectroscopy.  We're going to build a data-driven model of stars from the APOGEE survey and use it to measure the chemical abundances of stars throughout the Milky Way.  In the process, you'll learn how to build a data-driven model from scratch as well as work with neural networks.  

You'll also learn about stellar spectra, stellar chemistry, and stellar evolution.  This notebook provides broad background on stellar spectra including the basic properties of stars, why their spectra have features, the physics of stellar atmospheres, and how most contemporary stellar atmosphere models work.  Much of this background will be important for interpreting results from your data-driven model and sanity checking them.

Chemical tagging stars can help us learn what kind of environment (molecular cloud) they were born in, and help us identify stars that grew up in similar environments---in some cases well enough to co-identify members of the same birth group, even after they have been separated. It also helps us learn about the evolution of galaxies as early generations of stars generate heavier elements that aid the cooling and collapse of molecular clouds in subsequent generations.

## Some Science with Stellar Spectroscopy

* Galaxy Evolution and Chemical Cartography: <a href="https://arxiv.org/abs/1604.07435">Weinberg et al. 2017</a>, <a href="https://ui.adsabs.harvard.edu/#abs/1979ApJ...229.1046T/abstract">Tinsley 1978</a>
* Chemical Tagging: <a href="https://ui.adsabs.harvard.edu/#abs/2002ARA&A..40..487F/abstract">Freeman & Bland-Hawthorn 2002</a>, <a href="https://ui.adsabs.harvard.edu/#abs/2016ApJ...833..262H/abstract">Hogg et al. 2016</a>, <a href="https://arxiv.org/abs/1507.07563">Ting et al. 2016</a>
* Near-Field Cosmology, Galaxy Archaeology, First Stars: <a href="https://ui.adsabs.harvard.edu/#abs/2015ARA&A..53..631F/abstract">Frebel & Norris 2015</a>, <a href="https://ui.adsabs.harvard.edu/#abs/2018ApJ...856..138J/abstract">Ji et al 2018</a>, <a href="https://arxiv.org/abs/1901.05465">Simon 2019</a>
* <a href="https://cads.iiap.res.in/tools/stellarFluxCalc">Stellar Spectrum Calculator</a>


## Zeroth Order Description of stars
* A star is basically a very hot, luminous light bulb in the center (__stellar interior__) surrounded by cooler layers of gas (__stellar atmosphere__).
*  What we observe: a star's __spectrum__ is intensity or flux as a function of wavelength or freqency.
* A star's spectrum can be reasonably approximated as a __blackbody__  (i.e., Planck's function)
 * Assumes thermodynamic equilibrium
* # $B_{\lambda}(T) = \frac{2hc^2}{\lambda^5} \frac{1}{e^{hc/\lambda kT} - 1}$ $\big[\frac{erg}{s \ A \ str \ cm^2} \big]$
<img src='figs/blackbody.png' width=700 height=700></img>

## In practice, stars are not perfect blackbodies
* Emission and absoprtion features from different species of atoms and molecules
 * Due to quantum transitions of electrons
 * H$\alpha$ (n=3 to n=2), H$\beta$ (n=4 to n=2), etc.
 * Neutral vs. ionized: e.g., HI (atomic hydrogen), HeII (singly ionized helium)
* Not really in thermodynamic equilibriuim
* Example: compare Solar spectrum to blackbody

<img src='figs/solar_spectrum.jpg' width=700 height=700></img>

## Stars Referred to by Spectral "Types"
* Example of spectral types:
<img src='figs/specprf.gif' width=500 height=500></img>


* Both continuum (overall shape) and absoprtion / emission features change as a function of spectral type
 * T ~ 30,000 K -- highly ionized elements (O type)
 * T ~ 10,000K -- dominated by neutral H (A type)
 * T ~ 6,000K -- Ionized metal lines (G type)
 * T ~ 4,000K -- Neutral metals and molecules (M type)

### Connection between spectra type and CMD
<img src="https://sci.esa.int/documents/33580/35361/1567214809100-ESA_Gaia_DR2_HRD_Gaia_625.jpg" width=50%></img>


## Early history of stellar spectra
* In 19th century, stars classified by strength of Balmer lines of neutral hydrogen, gave rise to A-type, B-type, etc. -- led to confusing ordering later
 * 1880s, __Antonia Maury__ (Harvard) realized strength of all spectral lines changed continuously along a sequence, came up with a detailed classification system
 * 1901 -- __Annie Jump Cannon__ (Harvard) published simpler stellar classification system: OBAFGKM
 * 1925 -- __Cecilia Payne__ (Radcliff/Harvard) demonstrated that the Sun was mostly made of hydrogen and that effective temperatures were responsible for absorption / emission features
  * This was dismissed as nonsense at the time by Henry Norris Russell.  He later realized he was wrong and she was right, and published an admission of his mistake in 1929
  * Between 1911 and 1940, ~40,000 stars were classifed in the __Henry Draper (HD)__ catalog
  * __Maury's__ more detailed classification system form the sub-classes from 0 (hotter) to 9 (cooler). 
  * The Sun is a G2 star
 

## Physics of Stellar Spectra

* Spectral type is really a proxy for effective temperature, surface gravity, composition, and other related physics (rotation, turbulence, ...) 


* Basic physical ingredients for modeling a star's spectrum:
  * temperature, surface gravity, radius, chemical composition, rate of rotation, etc..


* __Surface gravity__
  * ## $g \propto \frac{M}{R^2}$
  * Giants have low surface gravity, dwarfs have high surface gravity
  * unit of acceleration (e.g., cm s$^{-2}$)
 
 

* __Effective temperature:__
  * ## $L = 4 \pi R^2 \sigma_{SB} T_{eff}^4$


* Making a model stellar atmosphere is a complicated physics problem
* Rely on (useful) some simplifying assumptions:
  * __Plane-Parallel Geometry__: only one spatial coordinate
  * __Hydrostatic Equilibrium__: most pressure comes from gas pressure
  * __No fine structres__: no starspots
  * __No magnetic fields__
  * __Stars don't rotate__

## Local Thermodynamic Equilibrium (LTE)
 
* __Blackbody assumption__  At the bottom of the stellar photosphere the optical depth to the surface is high enough to prevent escape of most photons. They are reabsorbed close to where they were emitted and radiation laws of BB apply. Distribution of particles in a star represent a range of temperatures.


* __Eqilibrium assumption cannot be true:__ stars have a net flow of energy (i.e., the stars radiate away energy)


* __NLTE:__ Higher layers deviate increasingly from Black Body case as this leakage becomes more significant. There is a continuous transition from near perfect local thermodynamic equilibrium (LTE) deep in the photosphere to complete __non-equilibrium (non-LTE)__ high in the atmosphere.


* __LTE__: Thermodynamic Equilibrium is applied to relatively small volumes of the model photosphere - volumes with dimensions of order unity in optical depth. This is why it's called __Local Thermodynamic Equilibrium (LTE)__ in practice.


* Distribution of photon energies: __Planck Law__ (Blackbody Relation)
* Distribution of kinetic energies: __Maxwell-Boltzmann Relation__
* Distribution among excitation levels: __Boltzmann Equation__
* Distribution among ionization states: __Saha Equation__



## Opacity

* Any process which removes photons from a beam of parallel light travelling through a gas will be called absorption, which includes scattering

* ## $dI_{\lambda} = -\kappa_{\lambda}\rho I_{\lambda} ds$

  * $dI_{\lambda}$: The change in intensity of light at some wavelength $(\lambda)$ as it travels through a gas
  * $ds$: the distance traveled
  * $I_{\lambda}$: the intensity of light
  * $\rho$: density of gas
  * $\kappa_{\lambda}$: The absorption coefficient, or opacity

* Opacity is a function of composition, density, & temperature.
* The detailed calculation of $\kappa_{\lambda}$ is a tough and extended problem within astrophysics, and cross-sections as a function of frequency for all sorts of atoms and molecules and even dust grains have to be calculated.
* The wavelength dependence shapes the continuous spectrum emitted by a star. It also has a major influence on the temperature structure of an atmosphere because the opacity controls how easily energy flows at a given wavelength.
* Where opacity is high, energy flow (flux) is low.
* Sources of opacity: Bound-Bound absorption, Bound-Free absorption, Free-Free absorption, Electron Scattering
* The primary source of opacity in most stellar atmospheres is: the photoionization of H$^-$ ions, but these become increasingly ionized for A, B stars, and then photoionization of H atoms and free-free absorption become the main sources.
* For O stars the main source is electron scattering, and the photoionization of He also contributes.
* Molecules can survive in cooler stellar atmospheres and contribute to bound-bound and bound-free opacities. The large numbers of molecular lines are an efficient impediment to the flow of photons.

## Radiative Transfer

* Describes the passge of light through a star's atmosphere:

* # $\frac{1}{\kappa_{\lambda}\rho} \, \frac{dI_{\lambda}}{ds} = \frac{dI_{\lambda}}{d\tau_{\lambda}}  = -I_{\lambda} + S_{\lambda}$
 
 * In an equilibrium (or steady-state) star there can be no change in the total energy contained within any layer of the stellar atmosphere. This means that mechanisms involving absorbing and emitting must be in balance throughout the star.
 
 * The intensity of light depends on the direction of travel to account for the net outflow of energy; although absorption and emission coefficients are the same for light traveling in all directions. They depend upon $T$ and $\rho$ in a complicated way.
 
 * To know anything about physical conditions in a stellar atmosphere $(T, \rho)$ we must know at what depth the spectral lines are formed, which means understanding and solving the radiative transfer equation.
 
 * Assumption of __Plane-Parallel__ atmosphere (atmospheric curvature $<<$ thickness of atmosphere) is very powerful and makes this problem lot easier
 
 * Almost all contemporary physical models of stellar atmosphere are 1D (i.e., plane-parallel)

## Create a Model Atmosphere & Spectra

### Stellar Atmosphere
 * Typical code input: $T_{\rm eff}$, log(g), Z (metallicity), turbulent velocity
 * Output: optical depth, temperature, gas pressure, electron pressue, mean molecular weight, opacity
 
### Synthetic Spectral Generation
 * The model "atmosphere", more specifically temp-presssure, temp-optical depth (called $T-\tau$) relationships are input into synthesis programs that take the physiscs, perform radiative transfer, and produce continuum + lines as a function of wavelength 
 
### Line Profiles

* A typical stellar spectrum, especially for cooler stars, contains thousands of lines, from many different chemical elements. Spectral lines have a variety of widths for which there are several causes, some internal to the atom, others external, and each produces a characteristic profile. Some types of profile have a broad core and small wings; others have a narrow core and extensive, broad wings.
 
* Line Widths have three main components: 
 * __Natural broadening__ (Lorentzian profile, very narrow)
 * __Doppler broadening__ (Maxwell-Boltzmann distribution, Gaussian) 
 * __Pressure (collisional) broadening__ (Lorentzian profile)
 
* Other, smaller effects: Zeeman splitting (strong magnetic effects on atoms), hyperfine (nuclear magnetic dipole)

<img src='figs/lineprofile.gif'></img>

## Relative Strength of Spectral Lines

<img src="figs/spectraltype.gif" width=800 height=800></img>

## A Few Words on Spectrographs

* Schematic example of a spectograph below
* Collimator makes incoming beam parallel
* Grating disperses the light by wavelength
* CCD records the flux at each wavelength

<img src="figs/telescope.png" width=500 height=500></img>
* __Spectroscopic resolution__: If spectrograph can barely distinguish two spectral features at wavelengths $\lambda$ and $\lambda + d\lambda$, its spectral resolution is: $R = \lambda/d\lambda$

* Note that $R$ can, and usually does, vary as a function of wavelength

* Nomenclature for what counts are high/medium/low resolution spectra varies quite a bit.  Generally:
 * __Low Resolution__: $R\ \lesssim$ 5000
 * __Medium Resolution__: $R \sim$ 5000-10000
 * __High Resolution__: $R \gtrsim 10^4$
 * Highest Resolution <a href=http://bass2000.obspm.fr/solar_spect.php>solar spectrum</a>: $R \sim 10^5 - 10^6$
 * In some sense, photometry can be thought of as really low resolution spectroscopy.  For example, a V-band filter of 1000 A would yields $R\sim 5000 / 1000 \sim 5$.